In [1]:
import pandas as pd
import numpy as np
import ast
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

In [8]:
user_item = pd.read_csv('../Datasets/user_items_cleaned.csv.gz')
steam_games = pd.read_csv ('../Datasets/steam_games_cleaned.csv.gz')
user_reviews = pd.read_csv ('../Datasets/user_review_cleaned.csv.gz')

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [54]:
def PlayTimeGenre(genero: str):
    merged_df = pd.merge(user_item, steam_games, left_on='item_name', right_on='app_name', how='inner')
    genre_df = merged_df[merged_df['genres'].apply(lambda x: genero in x)]
    grouped_df = genre_df.groupby('year')['playtime_forever'].sum()
    max_playtime_year = grouped_df.idxmax()
    result = {"Año de lanzamiento con más horas jugadas para Género X": max_playtime_year}
    return result


In [56]:
genre_result = PlayTimeGenre("FPS")
genre_result

{'Año de lanzamiento con más horas jugadas para Género X': 2012}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}



In [11]:
merged_df = pd.merge(user_item, steam_games, left_on='item_id', right_on='id')

In [12]:
def UserForGenre(genero):
    merge = merged_df
    
    usuario_agrupado = merge[['user_id','playtime_forever','year']].groupby(['user_id','year']).sum()
    usuario_agrupado_1 = usuario_agrupado['playtime_forever'].idxmax()
    genero = merge[merge['genres'].apply(lambda x: genero in x)]
    df_usuario_genero = merged_df[(merged_df['user_id'] == usuario_agrupado_1 [0])]
    poranio = df_usuario_genero.groupby('year')[['playtime_forever']].sum().reset_index().rename(columns={'year':'Año', 'playtime_forever': 'Horas jugadas'}).to_dict(orient='records')
    
    dicc = {
        'Uruario con mas horas jugadas por genero': usuario_agrupado_1 [0],
        'Horas Jugadas': poranio
    }

    return dicc
UserForGenre('Action')

{'Uruario con mas horas jugadas por genero': 'REBAS_AS_F-T',
 'Horas Jugadas': [{'Año': 1996, 'Horas jugadas': 0.0},
  {'Año': 1998, 'Horas jugadas': 1.0},
  {'Año': 1999, 'Horas jugadas': 225.0},
  {'Año': 2000, 'Horas jugadas': 0.0},
  {'Año': 2001, 'Horas jugadas': 11.0},
  {'Año': 2002, 'Horas jugadas': 1.0},
  {'Año': 2003, 'Horas jugadas': 1863.0},
  {'Año': 2004, 'Horas jugadas': 9102.0},
  {'Año': 2005, 'Horas jugadas': 3293.0},
  {'Año': 2006, 'Horas jugadas': 1847.0},
  {'Año': 2007, 'Horas jugadas': 10426.0},
  {'Año': 2008, 'Horas jugadas': 5033.0},
  {'Año': 2009, 'Horas jugadas': 61722.0},
  {'Año': 2010, 'Horas jugadas': 72958.0},
  {'Año': 2011, 'Horas jugadas': 186082.0},
  {'Año': 2012, 'Horas jugadas': 267560.0},
  {'Año': 2013, 'Horas jugadas': 403401.0},
  {'Año': 2014, 'Horas jugadas': 478036.0},
  {'Año': 2015, 'Horas jugadas': 1093887.0},
  {'Año': 2016, 'Horas jugadas': 1317392.0},
  {'Año': 2017, 'Horas jugadas': 35067.0}]}

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



In [52]:
import pandas as pd

def UsersRecommend(year: int):
    try:
        # Cargar el DataFrame desde el archivo CSV
        function3 = pd.read_csv('../Datasets/function3.csv')

        # Filtrar el DataFrame según las condiciones dadas
        filtered_df = function3[(function3['recommend'] == 1) & (function3['sentiment_analysis'].isin([1, 2])) & (function3['year'] == year)]

        # Agrupar por nombre de la aplicación y contar las recomendaciones
        game_recommendations = filtered_df.groupby('app_name')['recommend'].count().sort_values(ascending=False)

        # Obtener el top 3 de juegos recomendados como un diccionario
        top_3_games = game_recommendations.head(3).to_dict()

        # Crear el resultado con el formato deseado
        result = {"year": year, "top_3_games": [{"puesto {}".format(i + 1): {game}} for i, (game, _) in enumerate(top_3_games.items())]}

        return result

    except Exception as e:
        return {"Error": str(e)}




In [53]:
# Ejemplo de uso
UsersRecommend(2012)

{'year': 2012,
 'top_3_games': [{'puesto 1': {'Counter-Strike: Global Offensive'}},
  {'puesto 2': {'Borderlands 2'}},
  {'puesto 3': {'PlanetSide 2'}}]}

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



In [50]:
def UsersNotRecommend(year: int):
    try:
        # Cargar el DataFrame desde el archivo CSV
        function3 = pd.read_csv('../Datasets/function3.csv')

        # Filtrar el DataFrame según las condiciones dadas
        filtered_df = function3[(function3['recommend'] == 0) & (function3['sentiment_analysis'].isin([0, -1])) & (function3['year'] == year)]

        # Agrupar por nombre de la aplicación y contar las no recomendaciones
        game_not_recommendations = filtered_df.groupby('app_name')['recommend'].count().sort_values(ascending=False)

        # Obtener el top 3 de juegos menos recomendados como un diccionario
        top_3_not_recommended = game_not_recommendations.head(3).to_dict()

        # Crear el resultado con el formato deseado
        result = {"year": year, "top_3_not_recommended": [{"puesto {}".format(i + 1): {game}} for i, (game, _) in enumerate(top_3_not_recommended.items())]}

        return result

    except Exception as e:
        return {"Error": str(e)}




In [51]:
UsersNotRecommend(2012)

{'year': 2012,
 'top_3_not_recommended': [{'puesto 1': {'Counter-Strike: Global Offensive'}},
  {'puesto 2': {'Realm of the Mad God'}},
  {'puesto 3': {'Chivalry: Medieval Warfare'}}]}

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [48]:
def sentiment_analysis(year: int):
    try:
        function3 = pd.read_csv('../Datasets/function3.csv.gz', compression='gzip')
        filtered_df = function3[function3['year'] == year]
        sentiment_counts = filtered_df.groupby('sentiment_analysis').size().reset_index(name='count')
        result = {
            'Negative': int(sentiment_counts.loc[sentiment_counts['sentiment_analysis'] == 0, 'count'].values[0]) if 0 in sentiment_counts['sentiment_analysis'].values else 0,
            'Neutral': int(sentiment_counts.loc[sentiment_counts['sentiment_analysis'] == 1, 'count'].values[0]) if 1 in sentiment_counts['sentiment_analysis'].values else 0,
            'Positive': int(sentiment_counts.loc[sentiment_counts['sentiment_analysis'] == 2, 'count'].values[0]) if 2 in sentiment_counts['sentiment_analysis'].values else 0
        }

        return result

    except Exception as e:
        return {"Error": str(e)}


In [49]:
sentiment_analysis(2010)

{'Negative': 216, 'Neutral': 717, 'Positive': 852}

Si es un sistema de recomendación item-item:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.


Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.